# Wepick 데이터를 twiceSpark1의 ES에 벌크 인서트

-  환경: ElasticSearch 
-  테스트 머신: twiceSpark1

In [15]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import parallel_bulk, bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import random
import numpy as np
import sys
import pandas as pd
import csv

In [3]:
### twiceSpark1: 10.102.50.47
es_url = 'twiceSpark1:9200'

In [4]:
es = elasticsearch.Elasticsearch(es_url)

### wepick_data

format:
deal_id, user_id, slot_id, timestamp, impression_time, label(always one)

In [10]:
data_path = r'd:\WMIND\temp\pos_dataset_small.csv'

In [11]:
def make_wepick_documents2(file_path):
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            source = {}
            v, u, slot, imp_dt = int(row[0]), int(row[1]), int(row[2]), row[4]         
            source['u'] = u
            source['v'] = v
            source['slot'] = slot
            source['rgtime'] = pd.to_datetime(row[3], unit='ms').to_pydatetime()
            doc = {
                #"_op_type":'create',
                "_index":"wepick_seq",
                "_type":"doc",
                #"_id": u,
                "_source": source
            }
            yield(doc)

In [12]:
for x in make_wepick_documents2(data_path):
    print(x)
    break

{'_type': 'doc', '_source': {'v': 3473871, 'u': 10000015, 'slot': 24, 'rgtime': datetime.datetime(2018, 3, 29, 10, 16, 2, 191000)}, '_index': 'wepick_seq'}


In [13]:
es.indices.get_mapping(index='wepick_seq')

{'wepick_seq': {'mappings': {'doc': {'properties': {'rgtime': {'type': 'date'},
     'slot': {'type': 'long'},
     'u': {'type': 'long'},
     'v': {'type': 'long'}}}}}}

In [16]:
start = time.time()
total_time = 0
bulk(es, make_wepick_documents2(data_path))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 200.9969413280487 sec


In [15]:
res = es.search(index='wepick_seq', body={'query': {'match':{'u':'1000007'}}})

In [14]:
type(res['hits']['hits'][0]['_source']['hist'][0]['rgtime'])

KeyError: 'hist'

In [ ]:
res